In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('PWM20-21data.txt')
winter2 = pd.read_csv ('PWM21-22data.txt')
winter3 = pd.read_csv ('PWM22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [6]:
precip_days3[332:400]

,station,datetime,precip(mm),wxcodes
1457,PWM,2023-01-20 06:42:00,2.79,SN FZFG
1458,PWM,2023-01-20 06:51:00,3.56,SN FZFG
1459,PWM,2023-01-20 07:47:00,3.56,-SN BR
1460,PWM,2023-01-20 07:51:00,3.56,-SN BR
1461,PWM,2023-01-20 07:59:00,0.0001,-SN BR
...,...,...,...,...
1585,PWM,2023-01-23 06:15:00,1.02,-SN BR
1586,PWM,2023-01-23 06:37:00,1.27,-SN BR
1587,PWM,2023-01-23 06:51:00,1.52,-SN BR
1588,PWM,2023-01-23 07:00:00,0.0001,-SN BR


In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
pwm = LocationInfo(
    'Portland', 'Maine', 
    latitude= 43.6708,
    longitude= -70.2844,
    timezone='America/Connecticut'
)

pwm

LocationInfo(name='Portland', region='Maine', timezone='America/Connecticut', latitude=43.6708, longitude=-70.2844)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(pwm.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(pwm.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(pwm.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:55:35  2020-12-01 16:04:58  2020-12-01
1   2020-12-02 06:56:40  2020-12-02 16:04:40  2020-12-02
2   2020-12-03 06:57:44  2020-12-03 16:04:25  2020-12-03
3   2020-12-04 06:58:46  2020-12-04 16:04:13  2020-12-04
4   2020-12-05 06:59:47  2020-12-05 16:04:03  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:25:44  2021-02-24 17:23:27  2021-02-24
86  2021-02-25 06:24:07  2021-02-25 17:24:45  2021-02-25
87  2021-02-26 06:22:29  2021-02-26 17:26:03  2021-02-26
88  2021-02-27 06:20:50  2021-02-27 17:27:21  2021-02-27
89  2021-02-28 06:19:10  2021-02-28 17:28:38  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:55:19  2021-12-01 16:05:02  2021-12-01
1   2021-12-02 06:56:25  2021-12-02 16:04:44  2021-12-02
2   2021-12-03 06:57:29  2021-12-03 16:04:28  2021-12-03
3   2021-12-04 06:58:31  2021-12-04 16:04:15  2021-12-04
4   2021-12-05 06:59:33  2021-12-05 16:04:05  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:26:08  2022-02-24 17:23:08  2022-02-24
86  2022-02-25 06:24:30  2022-02-25 17:24:26  2022-02-25
87  2022-02-26 06:22:53  2022-02-26 17:25:44  2022-02-26
88  2022-02-27 06:21:14  2022-02-27 17:27:02  2022-02-27
89  2022-02-28 06:19:34  2022-02-28 17:28:20  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:55:03  2022-12-01 16:05:07  2022-12-01
1   2022-12-02 06:56:09  2022-12-02 16:04:48  2022-12-02
2   2022-12-03 06:57:13  2022-12-03 16:04:32  2022-12-03
3   2022-12-04 06:58:16  2022-12-04 16:04:18  2022-12-04
4   2022-12-05 06:59:18  2022-12-05 16:04:07  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:26:31  2023-02-24 17:22:49  2023-02-24
86  2023-02-25 06:24:54  2023-02-25 17:24:07  2023-02-25
87  2023-02-26 06:23:16  2023-02-26 17:25:25  2023-02-26
88  2023-02-27 06:21:38  2023-02-27 17:26:43  2023-02-27
89  2023-02-28 06:19:58  2023-02-28 17:28:01  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PWM,2020-12-01 00:51:00,3.81,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
1,PWM,2020-12-01 01:51:00,1.52,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
2,PWM,2020-12-01 02:51:00,1.52,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
3,PWM,2020-12-01 03:51:00,3.05,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
4,PWM,2020-12-01 04:51:00,2.03,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
...,...,...,...,...,...,...,...
249,PWM,2021-02-27 17:51:00,1.52,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
250,PWM,2021-02-27 18:51:00,3.30,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
251,PWM,2021-02-27 19:51:00,3.81,RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
252,PWM,2021-02-27 20:51:00,2.79,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PWM,2021-12-02 10:51:00,0.0001,-RASN,2021-12-02,2021-12-02 06:56:25,2021-12-02 16:04:44
1,PWM,2021-12-02 11:51:00,0.2500,-RASN BR,2021-12-02,2021-12-02 06:56:25,2021-12-02 16:04:44
2,PWM,2021-12-02 12:51:00,2.0300,RA BR,2021-12-02,2021-12-02 06:56:25,2021-12-02 16:04:44
3,PWM,2021-12-02 13:51:00,2.2900,RA BR,2021-12-02,2021-12-02 06:56:25,2021-12-02 16:04:44
4,PWM,2021-12-02 14:51:00,1.5200,-RA BR,2021-12-02,2021-12-02 06:56:25,2021-12-02 16:04:44
...,...,...,...,...,...,...,...
268,PWM,2022-02-25 20:51:00,0.5100,-SN BR,2022-02-25,2022-02-25 06:24:30,2022-02-25 17:24:26
269,PWM,2022-02-25 21:51:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:24:30,2022-02-25 17:24:26
270,PWM,2022-02-25 22:51:00,0.2500,-SN,2022-02-25,2022-02-25 06:24:30,2022-02-25 17:24:26
271,PWM,2022-02-25 23:51:00,0.0001,-SN,2022-02-25,2022-02-25 06:24:30,2022-02-25 17:24:26


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PWM,2022-12-01 00:51:00,3.8100,RA BR,2022-12-01,2022-12-01 06:55:03,2022-12-01 16:05:07
1,PWM,2022-12-01 18:51:00,0.0001,-SN,2022-12-01,2022-12-01 06:55:03,2022-12-01 16:05:07
2,PWM,2022-12-03 14:51:00,0.0001,-RA,2022-12-03,2022-12-03 06:57:13,2022-12-03 16:04:32
3,PWM,2022-12-03 15:51:00,0.2500,-RA BR,2022-12-03,2022-12-03 06:57:13,2022-12-03 16:04:32
4,PWM,2022-12-03 16:51:00,0.7600,-RA BR,2022-12-03,2022-12-03 06:57:13,2022-12-03 16:04:32
...,...,...,...,...,...,...,...
301,PWM,2023-02-26 16:51:00,0.5100,-SN,2023-02-26,2023-02-26 06:23:16,2023-02-26 17:25:25
302,PWM,2023-02-26 17:51:00,0.5100,SN FZFG,2023-02-26,2023-02-26 06:23:16,2023-02-26 17:25:25
303,PWM,2023-02-26 18:51:00,0.7600,-SN BR,2023-02-26,2023-02-26 06:23:16,2023-02-26 17:25:25
304,PWM,2023-02-26 19:51:00,0.5100,-SN BR,2023-02-26,2023-02-26 06:23:16,2023-02-26 17:25:25


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,PWM,2020-12-01 07:51:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
8,PWM,2020-12-01 09:51:00,1.0200,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
9,PWM,2020-12-01 10:51:00,0.7600,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
10,PWM,2020-12-01 12:51:00,0.0001,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
11,PWM,2020-12-01 13:51:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
...,...,...,...,...,...,...,...
237,PWM,2021-02-20 10:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:32:04,2021-02-20 17:18:11
245,PWM,2021-02-27 13:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
246,PWM,2021-02-27 14:51:00,0.7600,-SNRA,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
247,PWM,2021-02-27 15:51:00,0.7600,-SNRA,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,PWM,2020-12-01 09:51:00,1.0200,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58,2020-12-01 07:55:35,2020-12-01 15:04:58
9,PWM,2020-12-01 10:51:00,0.7600,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58,2020-12-01 07:55:35,2020-12-01 15:04:58
10,PWM,2020-12-01 12:51:00,0.0001,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58,2020-12-01 07:55:35,2020-12-01 15:04:58
11,PWM,2020-12-01 13:51:00,0.2500,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58,2020-12-01 07:55:35,2020-12-01 15:04:58
12,PWM,2020-12-01 14:51:00,0.5100,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58,2020-12-01 07:55:35,2020-12-01 15:04:58
...,...,...,...,...,...,...,...,...,...
236,PWM,2021-02-20 09:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:32:04,2021-02-20 17:18:11,2021-02-20 07:32:04,2021-02-20 16:18:11
237,PWM,2021-02-20 10:51:00,0.0001,-SN,2021-02-20,2021-02-20 06:32:04,2021-02-20 17:18:11,2021-02-20 07:32:04,2021-02-20 16:18:11
245,PWM,2021-02-27 13:51:00,0.0001,-SN,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21,2021-02-27 07:20:50,2021-02-27 16:27:21
246,PWM,2021-02-27 14:51:00,0.7600,-SNRA,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21,2021-02-27 07:20:50,2021-02-27 16:27:21


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.8018530120481927
2021-2022: 1.1896485714285714
2022-2023: 1.2902959999999999


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,PWM,2020-12-01 00:51:00,3.81,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
1,PWM,2020-12-01 01:51:00,1.52,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
2,PWM,2020-12-01 02:51:00,1.52,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
3,PWM,2020-12-01 03:51:00,3.05,RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
4,PWM,2020-12-01 04:51:00,2.03,-RA BR,2020-12-01,2020-12-01 06:55:35,2020-12-01 16:04:58
...,...,...,...,...,...,...,...
249,PWM,2021-02-27 17:51:00,1.52,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
250,PWM,2021-02-27 18:51:00,3.30,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
251,PWM,2021-02-27 19:51:00,3.81,RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21
252,PWM,2021-02-27 20:51:00,2.79,-RA BR,2021-02-27,2021-02-27 06:20:50,2021-02-27 17:27:21


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_2336\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 1.0570069767441863
2021-2022: 1.7082060606060607
2022-2023: 0.5832459770114944


In [68]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'ME'

data.to_csv('pwm.csv', index=False)